# Bond Repo Cashflows

This notebook demonstrates how to access and use the **Bond Repo** and demonstrates advanced Cashflows analysis within the **LSEG Financial Analytics SDK**. 

**You will be able to:**
- Define and Price Bond Repo
- Analyze detailed valuation metrics and Cashflows

## Imports


Import the following necessary modules:

- `lseg_analytics.pricing.instruments.repo` - for Bond Repo instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import repo as rp

import pandas as pd
import json
import datetime as dt
from IPython.display import display

## 1- No exchange of interest payments
The basic case where payments are only at the start and the end of the contract **Purchase/Repurchase**.

In this case, a repo parameter called **"is_coupon_exchanged"** must be set to false.

### Data Preparation

In [2]:
# 1.Define an underlying Bond instrument
fixed_bond_definition = rp.BondDefinition(
    notional_ccy = "USD",
    issue_date = dt.datetime.strptime("2025-01-01", "%Y-%m-%d"),
    end_date = dt.datetime.strptime("2030-01-01", "%Y-%m-%d"),
    fixed_rate_percent = 2,
    interest_payment_frequency = rp.InterestPaymentFrequencyEnum.QUARTERLY,
    interest_calculation_method = rp.InterestCalculationMethodEnum.DCB_ACTUAL_ACTUAL

)

underlying_bond = rp.RepoUnderlyingContract(
        instrument_definition = fixed_bond_definition,
        instrument_type = "Bond"
)
print("Underlying Bond definition created")

# 2.Define Repo instrument
repo_definition = rp.RepoDefinition(
    buy_sell = rp.IPABuySellEnum.BUY,
    start_date = dt.datetime.strptime("2025-01-01", "%Y-%m-%d"),
    tenor = "1Y",
    repo_rate_percent = 2, # Override repo rate instead of implying from repo curve
    underlying_instruments = [underlying_bond],
    is_coupon_exchanged = False
)

# 3.Create the Repo Instrument from the defintion
repo_instrument = rp.RepoDefinitionInstrument (definition = repo_definition)
print("Repo Instrument definition created")

# 4. Configure pricing parameters, optional
pricing_params = rp.RepoPricingParameters(
    valuation_date = dt.datetime.strptime("2025-07-18", "%Y-%m-%d"),                
)
print("Pricing parameters configured")

Underlying Bond definition created
Repo Instrument definition created
Pricing parameters configured


### Request Execution

In [3]:
# Execute the calculation using the price function
try:
    # The 'definitions' parameter accepts a list of request items for batch processing
    response = rp.price(
    definitions = [repo_instrument],
    pricing_preferences = pricing_params
)
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing Execution Successful!")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

Pricing Execution Successful!


### Results Display

After showing the Repo decription and analytics, we notice that two cashflows are returned:
- The first cashflow is the purchase price paid by the buyer
- The second cashflow is the repurchase price received by the buyer

Which is showed in the last code block

In [4]:
# Access the description object
description = response.data.analytics[0].description
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentDescription": "Repo 1000000 'USD' 2.00% 2026-01-02 Haircut: 0% / Initial Margin: 1.0000",
    "dealCcy": "USD",
    "startDate": "2025-01-01T00:00:00Z",
    "endDate": "2026-01-02T00:00:00Z",
    "initialMarginPercent": 100.0,
    "nominalPledgeQuantity": 1000000.0,
    "numberOfCoupons": 0,
    "dayCountBasis": "Dcb_Actual_360",
    "discountCurveId": "IRCurve_USD-OISSOFR_2025-07-18T00:00:00",
    "settlementConvention": "0WD",
    "repoRatePercent": 2.0
}


In [5]:
# Access the pricing analysis object
pricing_analysis = response.data.analytics[0]["pricingAnalysis"]
# Convert to DataFrame
df_pricing_analysis = pd.DataFrame(list(pricing_analysis.items()), columns=["Fields", "Value"])
# Display the DataFrame
display(df_pricing_analysis.head(5))

,Fields,Value
0,haircutRatePercent,0.0
1,marketDataDate,2025-07-18T00:00:00Z
2,valuationDate,2025-07-18T00:00:00Z
3,purchasePrice,1043778.653199
4,repurchasePrice,1064713.37372


In [6]:
# Access to payments/cash-flows
payments = response.data.analytics[0].cashflows.cash_flows[0]["payments"]
df_payments = pd.DataFrame(payments)
display(df_payments)

,date,discountFactor,amount,currency,event,occurence
0,2025-01-01,0.999638,-1.043779e+06,USD,Principal,Historical
1,2026-01-02,0.980617,1.064713e+06,USD,Principal,Future


In [7]:
print("purchasePrice", response.data.analytics[0]["pricingAnalysis"]["purchasePrice"])
print("repurchasePrice", response.data.analytics[0]["pricingAnalysis"]["repurchasePrice"])

purchasePrice 1043778.65319949
repurchasePrice 1064713.37372049


## 2- Exchange of interest payments
In this case the parameter **"is_coupon_exchanged"** must be set to true. It's the value by default if the parameter is not overriden.

### Data Preparation

In [8]:
repo_definition.is_coupon_exchanged = True
repo_definition.tenor = "2Y" # tenor extended only to visualize several cashflows

### Request Execution

In [9]:
# Execute the calculation using the price function
try:
    # The 'definitions' parameter accepts a list of request items for batch processing
    response = rp.price(
    definitions = [repo_instrument],
    pricing_preferences = pricing_params
)
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing Execution Successful!")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

Pricing Execution Successful!


### Results Display

Several cashflows are returned:
- The first cashflow is the purchase price paid by the buyer
- The last cashflow is the repurchase price received by the buyer
- The intermediate cashflows are interest payments

In [10]:
# Access to payments/cash-flows
payments = response.data.analytics[0].cashflows.cash_flows[0]["payments"]
df_payments = pd.DataFrame(payments)
display(df_payments)

,date,discountFactor,amount,currency,event,occurence
0,2025-01-01,0.999638,-1.043779e+06,USD,Principal,Historical
1,2025-10-01,0.991032,4.962932e+03,USD,Interest,Future
2,2026-01-02,0.980617,4.962932e+03,USD,Interest,Future
3,2026-04-01,0.971386,4.962932e+03,USD,Interest,Future
4,2026-07-01,0.962628,4.962932e+03,USD,Interest,Future
5,2026-10-01,0.954356,4.962932e+03,USD,Interest,Future
6,2027-01-04,0.946265,1.045714e+06,USD,Principal,Future
